In [1]:
import pandas as pd
import numpy as np
import random
import tensorflow as tf
from os.path import isfile, join
import os
import ast
from stable_baselines import DQN
import sys
sys.path.append('..')
from modules import utils
from modules import former_constants as constants
from multiprocessing import Process
from stable_baselines.common.vec_env import DummyVecEnv
from modules.env import LupusEnv
import warnings
warnings.filterwarnings('ignore')

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



Invalid MIT-MAGIC-COOKIE-1 key/home/lmuyama/anaconda3/envs/stable_baselines_tf2_env/lib/python3.7/site-packages/stable_baselines/__init__.py:33: UserWarning: stable-baselines is in maintenance mode, please use [Stable-Baselines3 (SB3)](https://github.com/DLR-RM/stable-baselines3) for an up-to-date version. You can find a [migration guide](https://stable-baselines3.readthedocs.io/en/master/guide/migration.html) in SB3 documentation.
  "stable-baselines is in maintenance mode, please use [Stable-Baselines3 (SB3)](https://github.com/DLR-RM/stable-baselines3) for an up-to-date version. You can find a [migration guide](https://stable-baselines3.readthedocs.io/en/master/guide/migration.html) in SB3 documentation."


In [2]:
# SEED = constants.SEED
SEED = constants.SEED
random.seed(SEED)
np.random.seed(SEED)
os.environ['PYTHONHASHSEED']=str(SEED)
tf.set_random_seed(SEED)
tf.compat.v1.set_random_seed(SEED)
SEED

126

#### Reading the data

In [3]:
val_df = pd.read_csv('../new_data/val_set_constant.csv')
val_df.head()

,ana,fever,leukopenia,thrombocytopenia,auto_immune_hemolysis,delirium,psychosis,seizure,non_scarring_alopecia,oral_ulcers,...,proteinuria,biopsy_proven_lupus_nephritis,anti_cardioliphin_antibodies,anti_β2gp1_antibodies,lupus_anti_coagulant,low_c3,low_c4,anti_dsdna_antibody,anti_smith_antibody,label
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,1,0,0,0
4,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
X_val = val_df.iloc[:, 0:-1]
y_val = val_df.iloc[:, -1]
X_val, y_val = np.array(X_val), np.array(y_val)
X_val.shape, y_val.shape

((5600, 24), (5600,))

In [5]:
def get_steps(filename, prefix):
    try:
        return int(filename[len(prefix)+1:][:-10])
    except Exception as e:
        print(f'Filename: {filename}')
        print(f'Exception: {e}')

In [6]:
# def check_pathway(pathway):
#     try:
#         pathway = ast.literal_eval(pathway)
#     except:
#         pass
#     return len(pathway), pathway[0] 

In [7]:
# get_steps('dqn_66000000_steps.zip', 'dqn')

In [8]:
# item = 'best_weighted_pahm_model_9_1.zip'
# item2 = 'dueling_dqn_per_17000000_steps.zip'
# prefix = 'dueling_dqn_per'
# item.starts_with(prefix)

In [9]:
def validate_model(model_name, seed, steps, X_val, y_val, prefix):
    best_f1, best_acc, best_roc_auc, best_pathway_score, best_pahm_score, best_wpahm_score = -1, -1, -1, -1, -1, -1
    perf_list = []
    count = 0
    
    folder = f'../models/logs/{model_name}/missingness/0.3/biopsy_9/knn_imputer/default_mean_k_1/cleps/seed_{seed}_{steps}'    
    for item in os.listdir(folder):        
        if item.startswith(prefix): # & (get_steps(item, prefix) > 20000000:
            path = join(folder, item)
            if (isfile(path)) & (path.endswith('.zip')):
                count+=1
                if count%10 == 0:
                    print(count)
                model = utils.load_dqn(path)
#                     wpahm_score = utils.get_val_metrics(model, X_val, y_val)
                pathway_score, pahm_score, wpahm_score, acc, f1, roc_auc, min_length, avg_length, max_length, min_path, max_path = utils.get_val_metrics(model, X_val, y_val)
                perf_dict = {'steps': get_steps(item, prefix), 'pathway_score':pathway_score, 
                             'pahm_score':pahm_score, 'weighted_pahm_score':wpahm_score, 'acc':acc, 'f1':f1, 
                             'roc_auc':roc_auc, 'min_path_length':min_length, 'avg_length':avg_length, 
                             'max_length':max_length, 'min_path':min_path, 'max_path':max_path} 
                perf_list.append(perf_dict)
                if pathway_score > best_pathway_score:
                    best_pathway_score = pathway_score
                    model.save(f'{folder}/best_pathway_model')
                if pahm_score > best_pahm_score:
                    best_pahm_score = pahm_score
                    model.save(f'{folder}/best_pahm_model')
                if wpahm_score > best_wpahm_score:
                    best_wpahm_score = wpahm_score
                    model.save(f'{folder}/best_weighted_pahm_model_9_1')
                if acc > best_acc:
                    best_acc = acc
                    model.save(f'{folder}/best_acc_model')
                if f1 > best_f1:
                    best_f1 = f1
                    model.save(f'{folder}/best_f1_model')
                if roc_auc > best_roc_auc:
                    best_roc_auc = roc_auc
                    model.save(f'{folder}/best_roc_auc_model')

    val_df = pd.DataFrame.from_dict(perf_list) 
    try:
        val_df = val_df.sort_values(by=['steps'])
    except:
        pass
    val_df = val_df.reset_index(drop=True)
    val_df.to_csv(f'{folder}/validation_results.csv', index=False)
    return val_df          

In [10]:
# names = ['dqn', 'ddqn', 'dueling_dqn', 'dueling_ddqn', 'dqn_per', 'ddqn_per', 'dueling_dqn_per', 'dueling_ddqn_per']
# for name in names[1:]:
#     print(f'{name.upper()} STARTING!')
#     eval_df = validate_model(name, SEED, steps, X_val, y_val, name)
# # eval_df.head()

In [11]:
# models = ['dqn', 'ddqn', 'dueling_dqn', 'dueling_ddqn', 'dqn_per', 'ddqn_per', 'dueling_dqn_per', 
#'dueling_ddqn_per']
# models = ['dqn_per', 'ddqn_per', 'dueling_dqn_per', 'dueling_ddqn_per']
# models = ['dueling_dqn_per', 'dueling_ddqn_per']
steps =int(100e6)
# procs=[]
validate_model('dueling_dqn_per', SEED, steps, X_val, y_val, 'dueling_dqn_per')

Loading a model without an environment, this model cannot be trained until it has a valid environment.






Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Please use `layer.__call__` method instead.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where







The environment seed is [126]
Loading a model without an environment, this model cannot be trained until it has a valid environment.
The environment seed is [126]
Loading a model without an environment, this model cannot be trained until it has a valid environment.
The environment seed is [126]
Loading a model without an environment, this model cannot be trained until it has a valid environment.
The environment seed is [126]
Loading a model without an environment, this model cannot be trained until it has a valid environment.
The environment seed is [126]
Loading a model without an environment, this model cannot be trained until it has a valid envir

The environment seed is [126]
Loading a model without an environment, this model cannot be trained until it has a valid environment.
The environment seed is [126]
Loading a model without an environment, this model cannot be trained until it has a valid environment.
The environment seed is [126]
Loading a model without an environment, this model cannot be trained until it has a valid environment.
The environment seed is [126]
Loading a model without an environment, this model cannot be trained until it has a valid environment.
The environment seed is [126]
Loading a model without an environment, this model cannot be trained until it has a valid environment.
The environment seed is [126]
Loading a model without an environment, this model cannot be trained until it has a valid environment.
The environment seed is [126]
Loading a model without an environment, this model cannot be trained until it has a valid environment.
The environment seed is [126]
Loading a model without an environment,

The environment seed is [126]
Loading a model without an environment, this model cannot be trained until it has a valid environment.
The environment seed is [126]
Loading a model without an environment, this model cannot be trained until it has a valid environment.
The environment seed is [126]
Loading a model without an environment, this model cannot be trained until it has a valid environment.
The environment seed is [126]
Loading a model without an environment, this model cannot be trained until it has a valid environment.
The environment seed is [126]
Loading a model without an environment, this model cannot be trained until it has a valid environment.
The environment seed is [126]
Loading a model without an environment, this model cannot be trained until it has a valid environment.
The environment seed is [126]
Loading a model without an environment, this model cannot be trained until it has a valid environment.
The environment seed is [126]
100
Loading a model without an environm

,steps,pathway_score,pahm_score,weighted_pahm_score,acc,f1,roc_auc,min_path_length,avg_length,max_length,min_path,max_path
0,1000000,1.571366,0.000000,0.000000,0.000000,0.000000,50.000000,18.0,24.639643,25.0,"[low_c4, thrombocytopenia, joint_involvement, ...","[low_c4, thrombocytopenia, joint_involvement, ..."
1,2000000,2.035746,0.000000,0.000000,0.000000,0.000000,50.000000,18.0,24.548393,25.0,"[non_scarring_alopecia, anti_cardioliphin_anti...","[non_scarring_alopecia, anti_β2gp1_antibodies,..."
2,3000000,2.753023,0.000000,0.000000,0.000000,0.000000,50.000000,16.0,24.366250,25.0,"[joint_involvement, pericardial_effusion, low_...","[joint_involvement, pericardial_effusion, cuta..."
3,4000000,3.204649,0.000000,0.000000,0.000000,0.000000,50.000000,18.0,24.260714,25.0,"[delirium, leukopenia, non_scarring_alopecia, ...","[delirium, leukopenia, non_scarring_alopecia, ..."
4,5000000,4.502016,3.258756,2.669088,2.553571,4.617372,49.941773,17.0,23.938036,25.0,"[pleural_effusion, leukopenia, pericardial_eff...","[pleural_effusion, thrombocytopenia, leukopeni..."
...,...,...,...,...,...,...,...,...,...,...,...,...
95,96000000,18.558138,30.983070,66.717943,93.750000,93.752973,93.776288,12.0,20.084286,25.0,"[anti_dsdna_antibody, fever, low_c3, delirium,...","[anti_dsdna_antibody, fever, anti_smith_antibo..."
96,97000000,21.048186,34.356353,69.521542,93.428571,93.426948,93.431675,11.0,19.071429,25.0,"[anti_β2gp1_antibodies, ana, psychosis, thromb...","[anti_β2gp1_antibodies, non_scarring_alopecia,..."
97,98000000,19.970427,32.925745,68.449863,93.732143,93.732119,93.732091,9.0,19.490893,25.0,"[seizure, psychosis, non_scarring_alopecia, an...","[seizure, non_scarring_alopecia, psychosis, an..."
98,99000000,17.746882,29.791275,65.180403,92.714286,92.694965,92.723765,11.0,19.919107,25.0,"[fever, proteinuria, cutaneous_lupus, ana, del...","[fever, proteinuria, cutaneous_lupus, ana, del..."


In [12]:
# for name in models:
#     proc = Process(target=validate_model, args=(name, SEED, steps, X_val, y_val, name))
#     procs.append(proc)
#     proc.start()

In [13]:
# for proc in procs:
#     proc.join()
# print('All jobs completed and terminated successfully')